# Controllers for Solo12
This notebook will be used as a base to design controller to be executed on the Solo12 real quadruped robot.

In [1]:
import math
import time
from ipywidgets import interact
from tp2.meshcat_viewer_wrapper import *
%config Completer.use_jedi = False
import numpy as np
from numpy.linalg import norm,inv,pinv,svd,eig
import matplotlib.pylab as plt
plt.ion()
np.set_printoptions(precision=3,suppress=True)
from example_robot_data.robots_loader import Solo12Loader
import pinocchio as pin
Solo12Loader.free_flyer = False #Important, we are working with a fixed based model version (12dof)

Let's load solo12 and setup a viewer

In [2]:
robot = Solo12Loader().robot
viz = MeshcatVisualizer(robot)
#viz.initViewer(loadModel=True)
#viz.viewer.jupyter_cell()
q0 = robot.q0
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [ ]:
# Add a red box in the viewer
ballID = "world/ball"
viz.addSphere( ballID, 0.05, colors.red )
# Place the ball at the position ( 0.5, 0.1, 0.2 )
# The viewer expect position and rotation, apppend the identity quaternion
q_ball = [.35, 0.1, -0.3, 1, 0, 0, 0]
viz.applyConfiguration( ballID, q_ball )
K = 10.0
r = .05

Let us have some input control to vary the robot state (Front left foot - 3dof)

In [ ]:
def torqueController(q):
    robot_pose = robot.framePlacement(q,robot.model.getFrameId('FL_FOOT'))
    vec = q_ball[0:3]-robot_pose.translation
    dist = np.linalg.norm(vec)
    print(dist)
    if (dist <= r):
        fc = K*(dist-r)*vec/dist
    else:
        fc = np.zeros(3)
    robot.forwardKinematics(q0)
    robot.framesForwardKinematics(q0)
    robot.computeJointJacobians(q0)
    J = robot.getFrameJacobian(robot.model.getFrameId('FL_FOOT'),pin.LOCAL_WORLD_ALIGNED)[:3]
    tau = np.transpose(J)@fc
    print(tau)

In [8]:
class Motor:
    def __init__(self,dt,size):
        self.dt = dt
        #State variables
        self.q =  np.zeros(size)#position de l'axe
        self.dq = np.zeros(size) #vitesse de l'axe
        self.ddq = np.zeros(size) #acceleration de l'axe
        self.tau = np.zeros(size) #couple
        
        #Model Parameters
        self.Kv = 1.0
        self.J = 1.0
        #self.retard = 1
        
        #self.previous_i = [0.0 for i in range(self.retard)]

    def reset(self):
        '''Reset all the internal state variable'''
        self.q = np.zeros(size)
        self.dq = np.zeros(size)
        self.ddq = np.zeros(size)
        self.tau = np.zeros(size)
        
        
    def update(self,i):
        #todo
        self.tau = self.Kv * i
        self.ddq = 1/self.J*self.tau
        self.dq += self.ddq*self.dt
        self.q += self.dq*self.dt                      
    

In [55]:
class Controller_copy:
    def __init__(self,robot):
        self.robot = robot
        self.Kp = 10
        self.Kd = 8
    
    def compute(self, q_mes, dq_mes):
        i = np.zeros(12)
        q_d = q_mes[:6]
        dq_d = q_mes[:6]
        i[6:] = self.Kp*(q_d - q_mes[6:]) + self.Kd*(0 - dq_mes[6:])
        return i

def copyPosition(q_copy, mots):
    controller = Controller_copy(robot)
    mots.q[:6] = q_copy[:6]
    for j in range(200):
        i = controller.compute(q_copy, mots.dq)
        mots.update(i)
        q_copy = mots.q
    q = mots.q
    return q

In [74]:
q0 = np.zeros(12)
viz.display(q0)
motors = Motor(0.01,12)
@interact(HAA=(-1.0, 1.0, 0.01),HFE=(-1.0, 1.0, 0.01),KFE=(-1.0, 1.0, 0.01))
def change_q(HAA=0.,HFE=0.,KFE=0.):
    q0[0:3] = HAA,HFE,KFE
    print(q0[0:3])
    oMref_fl = robot.framePlacement(np.zeros(12),robot.model.getFrameId('FL_FOOT')).translation
    oMref_hl = robot.framePlacement(np.zeros(12),robot.model.getFrameId('HL_FOOT')).translation
    oMtarget_fl = robot.framePlacement(q0, robot.model.getFrameId('FL_FOOT')).translation
    flMtarget_fl = oMtarget_fl - oMref_fl
    oMtarget = oMref_hl + flMtarget_fl/2
    q = inverseKin(q0, oMtarget)
    viz.display(q)
    print(q)

interactive(children=(FloatSlider(value=0.0, description='HAA', max=1.0, min=-1.0, step=0.01), FloatSlider(val…

In [73]:
# %load -r 36-62 tp3/inverse_kinematics.py
def inverseKin(q,oMtarget):
    herr = [] # Log the value of the error between tool and goal.
# Loop on an inverse kinematics for 200 iterations.
    for i in range(1):  # Integrate over 2 second of robot life

    # Run the algorithms that outputs values in robot.data
        robot.forwardKinematics(q0)
        robot.framesForwardKinematics(q0)
        robot.computeJointJacobians(q0)
        o_Jtool3 = robot.getFrameJacobian(robot.model.getFrameId('HL_FOOT'),pin.LOCAL_WORLD_ALIGNED)[:3]
        #print(o_Jtool3)
        # Placement from world frame o to frame f oMtool
        oMtool = robot.framePlacement(q,robot.model.getFrameId('HL_FOOT'))

        # vector from tool to goal, in world frame
        o_TG = oMtool.translation-oMtarget
        # Control law by least square
        vq = -pinv(o_Jtool3)@o_TG
        q = pin.integrate(robot.model,q, vq * 1)
        #viz.display(q)
        time.sleep(1e-3)

        herr.append(o_TG) 
    return q
   # plt.plot(herr)

In [ ]:
q = q0

oMref_fl = robot.framePlacement(np.zeros(12),robot.model.getFrameId('FL_FOOT')).translation
oMref_hl = robot.framePlacement(np.zeros(12),robot.model.getFrameId('HL_FOOT')).translation
oMtarget_fl = robot.framePlacement(q, robot.model.getFrameId('FL_FOOT')).translation
flMtarget_fl = oMtarget_fl - oMref_fl
oMtarget = oMref_hl + flMtarget_fl/2
print(oMref_fl)
print(oMtarget_fl)
print(flMtarget_fl)
inverseKin(q0,q_ball[0:3])

# Pinocchio cheat code

Get the Jacobian of the Feet, expressed locally, aligned with the world:

In [ ]:
robot.forwardKinematics(q0)
robot.framesForwardKinematics(q0)
robot.computeJointJacobians(q0)
J = robot.getFrameJacobian(robot.model.getFrameId('FL_FOOT'),pin.LOCAL_WORLD_ALIGNED)[:3]
print (J)